# <font size= 5> Introduction
# <font size= 5>This Notebook implements the ICG algorithm for solving the 2-objective assortment optimization under the **Multinomial Logit (MNL) model**. The primary goal is to identify a set of candidate assortments for a given collection of products, and the optimal assortment must be included. The entire process is divided into three main parts:
# <font size= 5> 1.  **Algorithm Implementation**: Based on a geometric approach, we first generates a list of candidate assortments.
# <font size= 5> 2.  **Identify the Optimal Assortment**: By checking the objective function value of all candidates, we find the optimal assortment.
# <font size= 5> 3.  **Correctness Verification**: The algorithm's results are validated by comparing them against a **brute-force enumeration** of all possible assortments to ensure the identified efficient frontier is correct.

**<font size= 5> Libraries<font>**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from itertools import combinations

**<font size= 5> Functions<font>**

**<font size= 4> Functions used in ICG:**

**Basic functions used to compute utility:**
- **These functions compute fundamental metrics for a given assortment `S` based on the MNL model.**

In [2]:
#Given an assortment S, prob(S) returns a list of choice probablities for all products in S

# S (list or array): A list of product indices.
# v (array): An array of product attractiveness values (randomly generated in each instance), global constant in each instance
# rev (array): An array of product revenues (randomly generated in each instance), global constant in each instance
# v0 (float): The attractiveness value of the no-purchase option, global constant in each instance


# Calculates the choice probability for each product in a given assortment S.
# Returns an array of choice probabilities for each product in S.
def prob(S):                                  
    Temp_prob = S
    return v.take(Temp_prob)/(v0+(v.take(Temp_prob)).sum())

#Returns the expected revenue of assortment S
def Rev(S):
    Temp_Rev = S
    return (rev.take(Temp_Rev)*(v.take(Temp_Rev)/(v0+(v.take(Temp_Rev)).sum()))).sum()

#Given an assortment S and x-coordinate, L returns the function value of \Lscr_{k}(x) (the formula is specific to Example B.1 in the paper).
# \Lscr_{k}(x) = \sum_{i \in S_k} (r_i - r_i^2 * x)q_i(S_k)
def L(S,x):
    Temp_L = S
    return (rev.take(Temp_L)*(v.take(Temp_L)/(v0+(v.take(Temp_L)).sum()))).sum()-((rev.take(Temp_L)**2*(v.take(Temp_L)/(v0+(v.take(Temp_L)).sum()))).sum())*x

#Given product index k, ell returns the function value of \omega_k(x) (the formula is specific to Example B.1 in the paper).
def ell(k,x):
    return rev[k] - (rev[k]**2)*x

#Returns the variance in revenue given assortment S
def Var(S):
    Temp_Var = S
    return (rev.take(Temp_Var)**2*(v.take(Temp_Var)/(v0+(v.take(Temp_Var)).sum()))).sum() - ((rev.take(Temp_Var)*(v.take(Temp_Var)/(v0+(v.take(Temp_Var)).sum()))).sum())**2


- **Helper functions for the ICG algorithm. For each of them, the parameter k represents the $k$-th loops**

In [3]:
"""
N_active_list: Global list, N_active_list[k-1] is the active product set at step k-1
Cand_Assort: Global list, Cand_Assort[k-1] is the candidate assortment found at step k-1
gamma_array: Global array that stores all x-coordinate of breakpoints, gamma_array[k] = $\gamma_k$ 
"""

# Finds the set of product lines l_i that are "parallel" (i.e., with equal E[rev^2] value) to the current envelope L_{S_{k-1}}.
# Returns a set of indices of such products.
def Parallel(k):
    N_parallel = set()
    Temp_Active = N_active_list[k-1]  # Get the active product set from the previous step
    Temp_Cand = Cand_Assort[k-1]      # Get the candidate assortment from the previous step
    
    # Check if E[rev^2] for each active product i is approximately equal to E[rev^2] of the current assortment S_{k-1}
    for i in Temp_Active:
        if abs(rev[i]**2 - (rev.take(Temp_Cand)**2*prob(Temp_Cand)).sum()) < allowed_error:
            N_parallel.add(i)
    return N_parallel

# Returns the x-coordinate of the intersection of Lscr_{k-1} and omega_i
def Intersection(k,i):
# i: product index
    Temp_Cand = Cand_Assort[k-1]
    return (rev[i]-Rev(Temp_Cand))/(-((rev.take(Temp_Cand)**2*prob(Temp_Cand)).sum())+rev[i]**2)

def Gamma_Update(k):
    """
    Computes the x-coordinate of the next breakpoint, gamma_k, and the set of product lines, N_update,
    that intersect with L_{S_{k-1}} at that point. This breakpoint is the one with the smallest x-coordinate
    among all future intersections.

    Returns:
            Output (list): [gamma_k, N_update], where gamma_k is the next breakpoint and N_update is the set of
            product indices that form the new upper envelope at that point.
    """

    Output = []
    Temp_N_Update = set()
    Temp_Gamma = 1e10    # Initialize with a large number to find the minimum
    Temp_index = 0
    Temp_Active = N_active_list[k-1]  # Get the current set of active products
    
    # Iterate through all non-parallel active product lines
    for i in Temp_Active.difference(Parallel(k)):
        # Find the leftmost intersection point to the right of the current breakpoint gamma_{k-1}
        if Intersection(k,i) - gamma_array[k-1] > allowed_error and Intersection(k,i)<Temp_Gamma:
            Temp_Gamma = Intersection(k,i)                                         
            Temp_index = i
    Temp_N_Update.add(Temp_index)
    Output.append(Temp_Gamma)
    Output.append(Temp_N_Update)   
    return Output

# The set unions with N^{update} in Step 3, which is the set of lines overlapping with Lscr_{k-1}(gamma_k)
# gamma: The x-coordinate of the current breakpoint.
def Reduction(k,gamma):
    A = set()
    for i in Parallel(k):
        # Check if the parallel line omega_i(gamma_k) has the same value as L_{S_{k-1}}(gamma_k)
        if abs(rev[i]-rev[i]**2*gamma - L(Cand_Assort[k-1],gamma_array[k])) < allowed_error:
            A.add(i)
    return A 

# Given gamma_k, returns the set of line indexs in N^{active} which follows that omega_i(gamma_k) > 0 at current interval
def Positive(k,gamma):
    A = set()
    for i in N_active_list[k-1]:
        if rev[i]-rev[i]**2*gamma > 0:
            A.add(i)
    return A

- **Functions used to compute the efficient frontier and check correctness**

In [4]:
'''
In the dual space, given two connected vertices S1,S2 of the convex hull, Line2Point(S1,S2) computes the slope of the line that 
defined by them. In the primal space, S1,S2 corresponds to two lines, and the slope corresponds to the x-axis of their intersection
'''
def Line2Point(S1,S2):
    Slope2x = (S2[1]-S1[1])/(S2[0]-S1[0])         
    return Slope2x

#Given an assortment S and an index j that determines lambda value, Obj returns the objective function value
def Obj(S,j):
    return Rev(S) - Var(S)*lambda_coeff[j]

## <font size = 5> Data Generation and Initialization
**Given a product set size, we randomly generate revenues of all products and necessary parameters to compute the attraction vector. We then reindex the products in descending order of revenue.**

In [5]:
# --- Parameters ---
n = 20        # number of products 
v0 = 1        # Attraction value of the no-purchase option
allowed_error = 1e-8 # Maximum error allowed in comparison

#Upperbound of parameters, where v = exp(alpha - beta* rev)
Upper_alpha = 10
Upper_beta = 1
Upper_rev = 10
#Global variables used to test speed and correctness
cand_number = []    # Number of candidate assortments generated in each run
optimal_number = [] # Number of optimal assortments on the efficient frontier in each run
time_number = []    # Execution time for each run
Correctness = []    # Correctness check result for each run (1 for correct, 0 for incorrect)             

#Generate randomized parameters
alpha = np.random.uniform(0,Upper_alpha,n)
beta = np.random.uniform(0,Upper_beta,n)
rev_Init = np.random.uniform(0,Upper_rev,n)

#Reorder products by revenue in descending order
start = time.process_time()                       #record the starting time
rev = np.array(sorted(rev_Init,reverse=True))
# Calculate attraction values 'v' based on the sorted revenues
v = np.exp(alpha-beta*rev)

**2. We then initialize the algorithm for k = 0 and compute Step1 of the algorithm**

In [6]:
#Initialization - Candidate assortments S_0 for gamma = 0
N = np.arange(n)                                 # The set of product indices {0,1,2,...,n-1}
Cand = [N[:i+1] for i in range(n)]               # Cand contains all revenue-ordered nested assortments: {0}, {0,1}, ...                     
S_0 = list(Cand[0])

# At gamma = 0, the objective is to maximize expected revenue Rev(S).
# Iterate through candidates to find the optimal S_0.
Gamma_0 = Rev(Cand[0])                           
for i in range(n):                               
    if Gamma_0 - Rev(Cand[i]) < allowed_error:
        Gamma_0 = Rev(Cand[i])
        S_0 = list(Cand[i])

# --- Initialize lists for storing iteration history ---

# Cand_Assort: Stores the candidate assortment S_k from each iteration. Cand_Assort[k] = S_k.
Cand_Assort = []
Cand_Assort.append(S_0)

#Alg Initialization
k = 1                                            
N_active = set(N)   # Initially, all products are active
# gamma_array: Stores the x-coordinates of all breakpoints. gamma_array[k] = gamma_k.
gamma_array = list(range(0,1))                    # The initial breakpoint is at 0
# N_active_list: Stores the active product set N_active for each iteration.
# N_active_list[k-1] = active set for the k-th iteration.
N_active_list = []
N_active_list.append(N_active)                    

# Calculate the first non-zero breakpoint, gamma_1
gamma_array.append(Gamma_Update(k)[0])  

In the code above, we created several important lists/arrays to store the output of each loop when we later iteratively run the algorithm, here we highlight the definitions of these variables:

- **Cand_Assort**: The list that stores all candidate assortment $S_k$ generated in each iteration(empty set is not included), Cand_Assort[k] = $S_k$
- **N_active_list**: The list that stores all set $N^{\sf active}$ for each iteration, N_active_list[k] is the set in k-th iteration
- **gamma_array**: The array that stores all x-coordinate of breakpoints, gamma_array[k] = $\gamma_k$ 

**3. We run Step2 and 3 iteratively to obtain all candidate assortments, utill Lscr intersects with the x-axis**. In each step, we follow the steps in the pseudo-code of ICG to update the sets and lists, and store $S_k$ and $N^{\sf active}$ for next iteration.

In [7]:
# Continue iterating as long as the upper envelope's value at the current breakpoint is positive
while (L(Cand_Assort[k-1],gamma_array[k]) > allowed_error):
    Sk_1 = set(Cand_Assort[k-1])                        # Candidate assortment from the previous step, S_{k-1}

    # --- Compute the components needed for the next step ---
    Temp_Gamma = Gamma_Update(k)[0]    # The new breakpoint, gamma_k
    Temp_N_Update = Gamma_Update(k)[1] # The product set intersecting the envelope at gamma_k
    Temp_Reduction = Reduction(k, Temp_Gamma) # The product set to be removed at gamma_k
    Temp_Positive = Positive(k, Temp_Gamma)  # The set of products with l_i > 0 at gamma_k
    
    # --- Update the candidate assortment and the active product set ---
    # Calculate the new candidate assortment: S_k = (S_{k-1} XOR N_update) - N_reduction
    Sk = (Sk_1.symmetric_difference(Temp_N_Update)).difference(Temp_Reduction)
    
    # N_remove are the products that were in S_{k-1} but are no longer in S_k
    N_remove = Sk_1.difference(Sk)                        
    
    # Update the active product set: N_active = {i in N_positive | i not in N_remove}
    N_active = Temp_Positive.difference(N_remove)

    # --- Store the results of this iteration ---
    Cand_Assort.append(list(Sk))
    N_active_list.append(N_active)

    # --- Prepare for the next iteration ---
    k = k + 1
    # Calculate the next breakpoint, gamma_{k+1}
    gamma_array.append(Gamma_Update(k)[0])

# After the loop terminates, the Cand_Assort list contains all generated candidate assortments.

When iteration stops, the latest **Cand_Assort** list contains all candidate assortments we generate.

**4. After obtaining all candidate assortments, we then work on identifying the efficient frontier of the given instance. The idea of the algorithm is given as follows:**

Since we have already discussed the property of the efficient frontier (i.e., a pointwise maximum function of all lines $(-{\sf Var}*\lambda+{\sf Rev})$, identifying efficient frontier is equivalent to identifying the upper envelope of all lines, each segment of which represents an candidate assortment. 

Then we note that all these lines can be converted to a point in the form of $(-{\sf Var}, -{\sf Rev})$ in the dual space, and the upper/lower envelope corresponds to the lower/upper convex hull of all converted points. The convex hull of a point set can be computed directly by using scipy library. 

Therefore, we just need to figure out which part of the convex hull belongs to the lower hull. Intuitively, we note that in the risk-sensitive case, our parameter $\lambda$ increases from 0 to $+\infty$, which implies that when $\lambda = 0$, the line with maximum ${\sf Rev}$ value is the uppermost line. Similarly, when $\lambda\rightarrow +\infty$, the line with maximum ${\sf -Var}$ value will be the uppermost line. After determining the start position and end position, we can quickly compute the efficient frontier (the ConvexHull function in scipy library includes all vertices of the convex hull in couterclock order in the 2-d plane). 

Besides, it is also worth noting that when computing the efficient frontier, if a segment of the efficient frontier intersects with x-axis, then the empty set becomes optimal after this intersection. So we compare the intersection of a line $L_k$ with the next line (i.e., the line which is adjcent in the list of lower hull vertices) with the root of $L_k$, if the root is smaller in x-coordinate, we break the loop and add 'empty set' as the last optimal assortment. Otherwise, we continue the loop. 

In [8]:
#Identify the efficient frontier
#Convert the lines L = -Var*lambda + Rev into points (-Var, -Rev)          
Points = np.array([[-Var(Cand_Assort[i]),-Rev(Cand_Assort[i])] for i in range(len(Cand_Assort))])

#Generate convex Hull, ConvexHull.vertices returns an array of vertex indexes in counterclock order
Hull = ConvexHull(Points)

# The lower hull starts at the vertex with the minimum -Rev value (i.e., max Rev, optimal for lambda=0)
# and ends at the vertex with the maximum -Var value (i.e., min Var, optimal for lambda -> inf).
Vertices = Points.take(Hull.vertices,axis=0)         # Get the coordinates of the hull's vertices   

#Find the start and end position in the array
Start_Position = np.where(Vertices[:,1] == np.min(Vertices[:,1]))[-1]
End_Position = np.where(Vertices[:,0] == np.max(Vertices[:,0]))[0]


# Constructing the lower hull using start and end position
# if start<end, then output the segment of a list, otherwise, traverse the list in a loop utill arriving at end_position

if Start_Position[0] > End_Position[0]:
    Optimal_Assort_Index = np.hstack((Hull.vertices[Start_Position[0]:] , Hull.vertices[:End_Position[0]+1]))                
else:
    Optimal_Assort_Index = Hull.vertices[Start_Position[0]:End_Position[0]+1]

# Get the list of optimal assortments using the extracted indices
Optimal_Assort_Temp = [Cand_Assort[i] for i in Optimal_Assort_Index]

# Generate the Efficient Frontier and Corresponding Lambda Intervals ---
Efficient_Frontier = [0]  # Breakpoints for lambda, starting from 0
Optimal_Assort = []       # The final list of optimal assortments
Optimal_Assort.append(Optimal_Assort_Temp[0])  # The first optimal assortment (for lambda=0)

# Iterate through the edges of the lower hull to calculate the lambda breakpoints where the optimal assortment changes
for i in range(len(Optimal_Assort_Temp)-1):
    BP = Line2Point(Points[Optimal_Assort_Index[i]],Points[Optimal_Assort_Index[i+1]])         #x-coord of breakpoint of L_i, L_{i+1}, euqal to the slope of the line covers D_i, D_{i+1}
    
    # Calculate the intersection of the current line L_i with the x-axis (xzero).
    # If the next breakpoint BP is to the right of xzero, it means the empty set becomes optimal before L_{i+1}.
    xzero = Points[Optimal_Assort_Index[i]][1]/(Points[Optimal_Assort_Index[i]][0])            #x_0 = -k/b = -Var/-Rev
    
    # If the switch happens before intersecting the x-axis, add the breakpoint and the next optimal assortment.
    if BP < xzero:                           
        Efficient_Frontier.append(BP)
        Optimal_Assort.append(Optimal_Assort_Temp[i+1])
    else:
     # Otherwise, the next breakpoint is the x-axis intersection, and the empty set becomes optimal.
        Efficient_Frontier.append(xzero)
        Optimal_Assort.append('Empty set')
        break

'''
For every efficient frontier, the empty set is always optimal when lambda is large enough, 
so we add empty set to the EF (and the root of the last line) if it has not been added
'''
if Optimal_Assort[-1] != 'Empty set':
    Efficient_Frontier.append(Points[Optimal_Assort_Index[len(Optimal_Assort_Temp)-1]][1]/(Points[Optimal_Assort_Index[len(Optimal_Assort_Temp)-1]][0]))
    Optimal_Assort.append('Empty set')
Efficient_Frontier.append('Infinity')  # The last interval extends to positive infinity

Finally, we compute the running time of the program and output the number of candidate assortments, the number of optimal assortments, the efficient frontier and the running time.

For the efficient frontier, we output lists of optimal assortments and intersections, for any element Optimal_Assort[k], it is optimal for $\lambda\in [{\sf Efficient~Frontier}[k],{\sf Efficient~Frontier}[k+1]]$

To conclude, all functions are computed based on global functions **rev[]**, **v[]**, **N_active_list[]**, **Cand_Assort[]**, we can run this program simply based on the input of $\sf rev$ and attraction vector $\sf v$, the latter one can either be computed using given parameters of $\alpha$, $\beta$ and $\sf rev$, or by known data. 

In [9]:
# Record the results of this experiment
cand_number.append(len(Cand_Assort))
optimal_number.append(len(Optimal_Assort))
end = time.process_time()
time_number.append(end - start)

# # Print the results
# print("All Candidate Assortments (Cand_Assort):")
# print(Cand_Assort)
# print("\nOptimal Assortments on the Efficient Frontier:")
# print(Optimal_Assort)          
# print("\nLambda Breakpoints of the Efficient Frontier:")
# print(Efficient_Frontier)   


**<font size = 5>Correctness Check<font>**

To check the correctness of our algorithm, we fix the lambda for each interval of the efficient frontier ($\lambda = 0.5*({\sf Efficient~Frontier}[k]+{\sf Efficient~Frontier}[k+1])$), and brutally computing the objective function values of all assortments (i.e., all subsets of {1,2,...,n}).

For each interval we brutally find the optimal assortment, and compare it to the the sets outputed in the Efficient Frontier algorithm part. The list **Correctness** stores the result of correctness in each run: if the results from brutal force and the Convex Hull method are the same, then the corresponding element in **Correctness** = 1, Otherwise = 0.

In [10]:
#Check correctness
#generate all subsets of N={1,2,...,n}
Subsets = []
for i in range(len(N)):
    for j in combinations(N,i+1):
        Subsets.append(j)

#Initialization & fix a lambda
Temp_optimal_index_list = []
Temp_optimal_index = []
lambda_coeff = [0.5*(Efficient_Frontier[i] + Efficient_Frontier[i+1]) for i in range(len(Efficient_Frontier)-2)]
lambda_coeff.append(Efficient_Frontier[-2]+1)        #List of fixed lambda, the last lambda equals to the intersection with x-axis plus 1


#Find the assortment achieving highest objective function value among all subsets
for j in range(len(lambda_coeff)):
    Temp_obj = 0
    Temp_optimal_index = []
    # Iterate through all subsets
    for i in Subsets:
        # Calculate the objective value for the current subset
        if Obj(i,j) > Temp_obj:
            Temp_obj = Obj(i,j)
            Temp_optimal_index = i
    if len(Temp_optimal_index) != 0:
        Temp_optimal_index_list.append(Temp_optimal_index)

# Compare the brute-force results with the algorithm's results
Tuple2List = [set(i) for i in Temp_optimal_index_list]
Alg_set = [set(i) for i in Optimal_Assort[:-1]]  # Exclude the final 'Empty set' from the algorithm's results

if Tuple2List == Alg_set:
    Correctness.append(1)
else: 
    Correctness.append(0)

#  Print statistics for the single run
print('The average number of candidate assortments is', sum(cand_number)/len(cand_number))
print('The average number of optimal assortments is', sum(optimal_number)/len(optimal_number))
print('The average running time is', sum(time_number)/len(time_number))
print('The correctness rate of the algorithm is', sum(Correctness)/len(Correctness))

The average number of candidate assortments is 37.0
The average number of optimal assortments is 6.0
The average running time is 0.078125
The correctness rate of the algorithm is 1.0


**<font size = 4>Integrated code for numerical experiments <font>**

In order to change the product set size or the number of iterations, one can modify parameters at the beginning of the code below, or the number of iterations.

In [11]:
#Parameters
n = 10        # number of products 
v0 = 1
allowed_error = 1e-9 # Maximum error allowed in comparison
np.random.seed(1) # Set random seed for reproducibility

#Upperbound of parameters
Upper_alpha = 10
Upper_beta = 1
Upper_rev = 10
#List of running time
cand_number = []
optimal_number = []
time_number = []
Correctness = []

#Test
for i in range(10):                    #Number of iterations

    #Generate randomized parameters
    alpha = np.random.uniform(0,Upper_alpha,n)
    beta = np.random.uniform(0,Upper_beta,n)
    rev_Init = np.random.uniform(0,Upper_rev,n)

    #Reorder products by revenue
    start = time.process_time()
    rev = np.array(sorted(rev_Init,reverse=True))
    v = np.exp(alpha-beta*rev)

    #Initialization - Candidate assortments S_0 for gamma = 0
    N = np.arange(n)                                 #The full set given n
    Cand = [N[:i+1] for i in range(n)]               #all n candidates                     
    S_0 = list(Cand[0])
    Gamma_0 = Rev(Cand[0])
    for i in range(n):
        if Gamma_0 - Rev(Cand[i]) < allowed_error:
            Gamma_0 = Rev(Cand[i])
            S_0 = list(Cand[i])

    #The list to store candidate assortments(empty set is not included), Cand_Assort[k]= S_k                
    Cand_Assort = []
    Cand_Assort.append(S_0)

    #Alg Initialization
    k = 1                                            
    N_active = set(N)
    gamma_array = list(range(0,1))                    #list of breakpoints
    N_active_list = []
    N_active_list.append(N_active)                    #N_active_list[k-1] = N_active at k-th iteration

    #First run of step 2
    gamma_array.append(Gamma_Update(k)[0])  

    while (L(Cand_Assort[k-1],gamma_array[k]) > allowed_error):
        Sk_1 = set(Cand_Assort[k-1])                        #S_{k-1}
        Temp_Gamma = Gamma_Update(k)[0]
        Temp_N_Update = Gamma_Update(k)[1]
        Temp_Reduction = Reduction(k,Temp_Gamma)
        Temp_Positive = Positive(k,Temp_Gamma)
        
        Sk = (Sk_1.symmetric_difference(Temp_N_Update)).difference(Temp_Reduction)
        N_remove = Sk_1.difference(Sk)
        N_active = Temp_Positive.difference(N_remove)
        Cand_Assort.append(list(Sk))
        N_active_list.append(N_active)
        k = k + 1
        gamma_array.append(Gamma_Update(k)[0])

    #Identify the efficient frontier
    #Convert the lines L = -Var*lambda + Rev into points (-Var, -Rev)          
    Points = np.array([[-Var(Cand_Assort[i]),-Rev(Cand_Assort[i])] for i in range(len(Cand_Assort))])
    #Generate convex Hull
    Hull = ConvexHull(Points)
    #Find the lower hull, which starts from the vertex with the min (-Rev)-value (i.e., the one at the 'bottom' of the hull), and ends at the vertex with the max (-Var) (i.e., the rightmost one)
    Vertices = Points.take(Hull.vertices,axis=0)                                  #array of vertices, in the form of (-Var, -Rev)
    Start_Position = np.where(Vertices[:,1] == np.min(Vertices[:,1]))[-1]
    End_Position = np.where(Vertices[:,0] == np.max(Vertices[:,0]))[0]
    #The indexes of optimal assortments in the Cand_Assort
    if Start_Position[0] > End_Position[0]:
        Optimal_Assort_Index = np.hstack((Hull.vertices[Start_Position[0]:] , Hull.vertices[:End_Position[0]+1]))
    else:
        Optimal_Assort_Index = Hull.vertices[Start_Position[0]:End_Position[0]+1]
    Optimal_Assort_Temp = [Cand_Assort[i] for i in Optimal_Assort_Index]
    #Generate EF, from 0 to +∞. For the focal line, compare its intersection with the next line and the x-axis to check if we need to break the loop.  
    Efficient_Frontier = [0]
    Optimal_Assort = []
    Optimal_Assort.append(Optimal_Assort_Temp[0])
    for i in range(len(Optimal_Assort_Temp)-1):
        BP = Line2Point(Points[Optimal_Assort_Index[i]],Points[Optimal_Assort_Index[i+1]])         #x-coord of breakpoint of L_i, L_{i+1}, euqal to the slope of the line covers D_i, D_{i+1}
        xzero = Points[Optimal_Assort_Index[i]][1]/(Points[Optimal_Assort_Index[i]][0])            #x_0 = -k/b = -Var/-Rev
        if BP < xzero:
            Efficient_Frontier.append(BP)
            Optimal_Assort.append(Optimal_Assort_Temp[i+1])
        else:
            Efficient_Frontier.append(xzero)
            Optimal_Assort.append('Empty set')
            break
    if Optimal_Assort[-1] != 'Empty set':
        Efficient_Frontier.append(Points[Optimal_Assort_Index[len(Optimal_Assort_Temp)-1]][1]/(Points[Optimal_Assort_Index[len(Optimal_Assort_Temp)-1]][0]))
        Optimal_Assort.append('Empty set')
    Efficient_Frontier.append('Infinity')
    #compute number of optimal assortments and running time
    cand_number.append(len(Cand_Assort))
    optimal_number.append(len(Optimal_Assort))
    end = time.process_time()
    time_number.append(end - start)

    #print(Cand_Assort)
    #print(Optimal_Assort)                  
    #print(Efficient_Frontier)   

    #Check correctness
    #generate all subsets of N={1,2,...,n}
    Subsets = []
    for i in range(len(N)):
        for j in combinations(N,i+1):
            Subsets.append(j)

    #Initialization & fix a lambda
    Temp_optimal_index_list = []
    Temp_optimal_index = []
    lambda_coeff = [0.5*(Efficient_Frontier[i] + Efficient_Frontier[i+1]) for i in range(len(Efficient_Frontier)-2)]
    lambda_coeff.append(Efficient_Frontier[-2]+1)        #List of fixed lambda, the last lambda equals to the intersection with x-axis plus 1


    #Find the assortment achieving highest objective function value among all subsets
    for j in range(len(lambda_coeff)):
        Temp_obj = 0
        Temp_optimal_index = []
        for i in Subsets:
            if Obj(i,j) > Temp_obj:
                Temp_obj = Obj(i,j)
                Temp_optimal_index = i
        if len(Temp_optimal_index) != 0:
            Temp_optimal_index_list.append(Temp_optimal_index)
    Tuple2List = [set(i) for i in Temp_optimal_index_list]
    Alg_set = [set(i) for i in Optimal_Assort[:-1]]
    if Tuple2List == Alg_set:
        Correctness.append(1)
    else: 
        Correctness.append(0)
        #To jump out when there is inconsistent assortment, add break
        #break

print('The average number of candidate assortments is', sum(cand_number)/len(cand_number))
print('The average number of optimal assortments is', sum(optimal_number)/len(optimal_number))
print('The average running time is', sum(time_number)/len(time_number))
print('The correctness rate of the algorithm is', sum(Correctness)/len(Correctness))

The average number of candidate assortments is 17.3
The average number of optimal assortments is 6.2
The average running time is 0.0171875
The correctness rate of the algorithm is 1.0
